<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM_mcfixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [1]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):
    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}


def plot_all( sample = None, model = None, labels = None):
    img = sample[:,:].squeeze().cpu().numpy()
    #img = img.T
    plt.imshow(img, cmap=plt.cm.gray_r)
    if model != None:
        with torch.no_grad():
          global numpoints
          sample = sample.unsqueeze(0).cuda()
          labels = labels.unsqueeze(0).cuda()

          #print('samplelabel',sample.shape,labels.shape)
          h0 = None
          c0 = None
          X = []
          Y = []
          for i in range(11):
            optimizer.zero_grad()
            if i == 0:
              out, h0, c0,o,output = model(sample,labels[:,0,:2],h0,c0,ind = i)
            else:
              out, h0, c0,o,output = model(sample,labels[:,0,:2],h0,c0,ind = i,o = o,output=output)
              
            X.extend(out.detach()[0,:,0].tolist())
            Y.extend(out.detach()[0,:,1].tolist())

          predres = len(X)

          s = [.001 for x in range(predres)]
          
          assert len(s) == predres
          c = ['red' for x in range(predres)]
          assert len(c) == predres
          #print("type",type(X))
          plt.scatter(X,Y,s = s,c = c)
        #plt.gca().add_artist(ascatter)
    else:
        #print(labels.shape)
        X = labels[:,0]
        Y = labels[:,1]
        #print(X.shape)
        #print(Y.shape)
        s = [.001 for x in range(numpoints)]
        #print(len(s))
        c = ['red' for x in range(numpoints)]
        #print(len(c))
        #ascatter = 
        plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
        #plt.gca().add_artist(ascatter)

class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,dataset, model):
        #model.setBatchSize(batch_size = 1)
        if model != None:
          model = model.eval()
        for i in range(100):
            sample, labels = dataset[i]
            plt.subplot(10,10,i+1)
            plot_all(sample = sample,model=model, labels = labels)
            plt.axis('off')

        plt.savefig(title,dpi=450)
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [2]:
for i in range(100):
  print(torch.sum(dataset[i][0]))

tensor(76.)
tensor(92.)
tensor(147.)
tensor(121.)
tensor(91.)
tensor(77.)
tensor(256.)
tensor(171.)
tensor(133.)
tensor(130.)
tensor(129.)
tensor(86.)
tensor(92.)
tensor(172.)
tensor(74.)
tensor(74.)
tensor(69.)
tensor(84.)
tensor(117.)
tensor(102.)
tensor(153.)
tensor(135.)
tensor(188.)
tensor(215.)
tensor(88.)
tensor(75.)
tensor(75.)
tensor(196.)
tensor(157.)
tensor(190.)
tensor(87.)
tensor(103.)
tensor(86.)
tensor(108.)
tensor(141.)
tensor(115.)
tensor(104.)
tensor(89.)
tensor(230.)
tensor(258.)
tensor(133.)
tensor(145.)
tensor(92.)
tensor(79.)
tensor(104.)
tensor(139.)
tensor(89.)
tensor(177.)
tensor(107.)
tensor(119.)
tensor(198.)
tensor(125.)
tensor(139.)
tensor(110.)
tensor(179.)
tensor(111.)
tensor(117.)
tensor(240.)
tensor(192.)
tensor(157.)
tensor(85.)
tensor(95.)
tensor(94.)
tensor(98.)
tensor(228.)
tensor(215.)
tensor(159.)
tensor(180.)
tensor(89.)
tensor(103.)
tensor(88.)
tensor(72.)
tensor(93.)
tensor(243.)
tensor(241.)
tensor(110.)
tensor(121.)
tensor(74.)
tensor(128.)
t

In [3]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 64
test_dataset = DonutDataset(length = 64)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=4)


x0_y0_r_xrf_yrf torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [4]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 64
dataset = DonutDataset(length = 64*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=4)


x0_y0_r_xrf_yrf torch.Size([1280, 1]) torch.Size([1280, 1]) torch.Size([1280, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [5]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 256
        self.num_layers = 2
        self.seq_len = 50
        self.lm_len = 20
        self.full_len = 1000
        self.longtermMem = None
        self.longtermMemP = None
        self.lstm_cell = nn.LSTM(self.hidden_size, self.hidden_size,self.num_layers)

        self.inject_image =  nn.Sequential(
            nn.Linear(2*1000+1024+3*self.hidden_size,self.hidden_size),
            nn.ReLU()
        )

        
        self.fc_out = nn.Sequential(
            nn.Linear(2+self.hidden_size*4,10000),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(10000,2),
            nn.Sigmoid()
        )
        self.hi_1 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )
        self.hi_2 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        
        )

        self.ci_1 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )
        self.ci_2 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )

        self.alpha_1 = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long1 = nn.Sequential(
            nn.Linear(self.hidden_size,self.lm_len),
            nn.Softmax()
        )
        

        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()
        
        
        

            
        
    def forward(self, features,p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        assert features != None
        #assert captions != None
        #print('featursescaptions2',features.shape,captions.shape)
        #assert features.shape[0]==captions.shape[0]
        assert len(features.shape) == 3
        self.track +=1
        batch_size = features.shape[0]
        

        #img_emb = torch.empty((batch_size, captions.size(1), self.hidden_size)).cuda()
        
        p0 = torch.flatten(p0[:,:2],start_dim=1)
        

        if o == None:
          out = p0
          self.longtermMem = torch.zeros(batch_size,self.lm_len,self.hidden_size).cuda()
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          self.longtermMemP = torch.zeros(batch_size,1000,2).cuda()
        else:
          out = o

        
        if h0 == None and c0 == None:
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,torch.zeros(batch_size,self.hidden_size).cuda(),torch.zeros(batch_size,self.hidden_size).cuda()],dim=1)
          h_0_1 = self.hi_1(combin_init)  
          h_0_2 = self.hi_2(combin_init)  
          c_0_1 = self.ci_1(combin_init)  
          c_0_2 = self.ci_2(combin_init)  
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        #else:
          #combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),h0[0,:,:]],dim=1)
          #h_0_1 = self.hi_1(combin_init)  
          
          #combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),h0[1,:,:]],dim=1)
          #h_0_2 = self.hi_2(combin_init)  
          
          #combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),c0[0,:,:]],dim=1)
          #c_0_1 = self.ci_1(combin_init)  
          
          #combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),c0[1,:,:]],dim=1)
          #c_0_2 = self.ci_2(combin_init)
          
          #h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          #c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        
        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None

        for t in range(self.seq_len):
        
            # for the first time step the input is the feature vector
            if output == None:
              output = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1 = torch.zeros(batch_size,self.hidden_size).cuda()
            
            #print('1',torch.flatten(features,start_dim=1).shape,torch.flatten(self.longtermMemP,start_dim=1).shape,output.squeeze().shape,)
            
            combin_lstm = torch.cat([torch.flatten(features,start_dim=1),torch.flatten(self.longtermMemP,start_dim=1),output.view(batch_size,self.hidden_size),outesread1,outesreadl1],dim=1)
            combin_lstm = self.inject_image(combin_lstm)
            
            a1 = self.alpha_1(combin_lstm)
            #print('a1',a1.shape,a1.unsqueeze(2).shape,self.embs.shape,outesread1.shape)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            #print('122',self.longtermMem.shape,outesread1.shape)
            #print('ind',ind)
            self.longtermMem[:, ind, :] = (outesread1).detach()
            
            al1 = self.alpha_long1(combin_lstm)
            outesreadal1 = self.longtermMem.clone()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))

            outputs[:, t, :] = out
            self.longtermMemP[:,50*ind+t,:] = out.detach()
            
            self.embs[:, t, :] = output.view(batch_size,self.hidden_size).clone().detach()
            

            
            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            self.longtermMem[:, ind, :] = (outesread1).detach()
            
            al1 = self.alpha_long1(combin_lstm)
            outesreadal1 = self.longtermMem.clone()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            

            
            combOut = torch.cat([combin_lstm, output.view(batch_size,self.hidden_size),out,outesread1,outesreadl1],dim=1)#,outesread],dim=1)
            

            out = self.fc_out(combOut)
            out = out*32.0
            # build the output tensor
        
        #self.embs = self.embs.detach().clone()
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions


In [6]:
#torch.stack([torch.zeros(64,10),torch.ones(64,10)],dim = 0)[0,:,:]

In [7]:
a = torch.ones(64,256,50)
b = torch.from_numpy(np.random.randn(50))
c = a*b
print(c[0,1,:])
print(c[1,1,:])

tensor([-1.2895,  1.4533, -0.1549,  0.2255, -0.1193, -0.2180, -0.2221,  0.5888,
        -0.4340,  0.4630, -0.5141,  0.5818,  0.7152, -1.3863,  0.5177,  0.4948,
         0.4553,  1.7217, -0.9780,  0.0247,  0.2858,  0.9300, -0.3600, -0.0984,
        -0.2104,  2.1738, -0.6221, -0.4916,  1.4020, -0.7003, -1.8666,  0.8654,
        -0.3302,  0.6693, -0.0552, -0.4083,  0.4083, -1.2317, -1.4357,  0.6407,
        -0.3556,  0.4375,  0.3853,  0.0066, -1.5084,  0.8754,  0.9280, -0.1872,
         1.4183,  0.2169], dtype=torch.float64)
tensor([-1.2895,  1.4533, -0.1549,  0.2255, -0.1193, -0.2180, -0.2221,  0.5888,
        -0.4340,  0.4630, -0.5141,  0.5818,  0.7152, -1.3863,  0.5177,  0.4948,
         0.4553,  1.7217, -0.9780,  0.0247,  0.2858,  0.9300, -0.3600, -0.0984,
        -0.2104,  2.1738, -0.6221, -0.4916,  1.4020, -0.7003, -1.8666,  0.8654,
        -0.3302,  0.6693, -0.0552, -0.4083,  0.4083, -1.2317, -1.4357,  0.6407,
        -0.3556,  0.4375,  0.3853,  0.0066, -1.5084,  0.8754,  0.9280, -

In [8]:
model = DecoderRNN().cuda()

In [9]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [10]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [11]:
model = model.train()
for seql in range(3,12,4):
  #seql = 19
  for epoch in range (2):
    loss = None
    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 3
tensor(21.3183, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(45.5222, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(10.1346, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(11.9310, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(5.8972, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(6.5005, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.9395, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(7.1837, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(5.0093, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(6.1507, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.8787, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.7161, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.5582, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.4280, device='cuda:0', grad_fn=<Mean

In [12]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [13]:

model = model.train()
#for seql in range(15,20,4):
seql = 11
for epoch in range (1):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 11
tensor(1.9591, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8255, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.9413, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8264, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7185, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7037, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6496, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6783, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7070, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6712, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7752, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8089, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7852, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8735, device='cuda:0', gra

In [14]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [15]:

model = model.train()
#for seql in range(15,20,4):
seql = 11
for epoch in range (1):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 11
tensor(1.8475, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6751, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7744, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6723, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6322, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8441, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7171, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8155, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6326, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.8466, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6849, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6895, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.5634, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4796, device='cuda:0', gra

In [16]:

#optimizer.zero_grad()


In [17]:
"""model = model.train()
for epoch in range (20):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(19):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      #print(x.shape,y.shape)
      if i == 0:
        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i)
      else:
        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i,o=o)
        
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)

      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch)
    print(loss)"""

'model = model.train()\nfor epoch in range (20):\n  loss = None\n  for x,y in loader_train:\n    h0 = None\n    c0 = None\n    for i in range(19):\n      x = x.cuda()\n      y = y.cuda()\n      \n      optimizer.zero_grad()\n      #print(x.shape,y.shape)\n      if i == 0:\n        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i)\n      else:\n        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i,o=o)\n        \n      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)\n      writer.add_scalar("Loss/train", loss, epoch)\n\n      loss.backward()\n      optimizer.step()\n    #print(y[:,0,:2])\n    print(\'epoch\',epoch)\n    print(loss)'

In [18]:
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 32
test_dataset = DonutDataset(length = 64*10)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=4)


x0_y0_r_xrf_yrf torch.Size([640, 1]) torch.Size([640, 1]) torch.Size([640, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [19]:
#model = model.eval()
model = model.eval()
with torch.no_grad():
  for epoch in range (1):
    loss = None
    for x,y in loader_test:
      h0 = None
      c0 = None
      seql = 11
      loss = torch.zeros(seql)
      for i in range(seql):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        
        #print(x.shape,y.shape)
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
        loss[i] = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
        print(loss[i])
        writer.add_scalar("Loss/val", loss[i],i, epoch)
        

      #print(y[:,0,:2])
      print('epoch',epoch)
      print('test loss', torch.mean(loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(88.1316)
tensor(5.0191)
tensor(2.8130)
tensor(1.6374)
tensor(1.4348)
tensor(1.6648)
tensor(1.6568)
tensor(1.8815)
tensor(1.8203)
tensor(2.4373)
tensor(2.0038)
epoch 0
test loss tensor(10.0455)
tensor(87.2928)
tensor(4.6011)
tensor(2.3368)
tensor(1.5830)
tensor(1.2786)
tensor(1.5725)
tensor(1.6518)
tensor(2.0848)
tensor(1.6403)
tensor(2.0822)
tensor(2.1918)
epoch 0
test loss tensor(9.8469)
tensor(87.2399)
tensor(4.6922)
tensor(2.4241)
tensor(1.6732)
tensor(1.3023)
tensor(1.3889)
tensor(1.3976)
tensor(1.9727)
tensor(1.9776)
tensor(2.0616)
tensor(2.2286)
epoch 0
test loss tensor(9.8508)
tensor(85.6565)
tensor(4.9377)
tensor(2.5447)
tensor(1.7685)
tensor(1.2545)
tensor(2.0183)
tensor(1.8809)
tensor(2.4164)
tensor(1.7244)
tensor(2.1611)
tensor(2.5248)
epoch 0
test loss tensor(9.8989)
tensor(85.8297)
tensor(4.3820)
tensor(2.2922)
tensor(1.4620)
tensor(1.1682)
tensor(1.5631)
tensor(1.5772)
tensor(1.8743)
tensor(1.9052)
tensor(2.3724)
tensor(2.7437)
epoch 0
test loss tensor(9.7427)
tens

In [20]:
writer.flush()
writer.close()


In [21]:
#%load_ext tensorboard

In [22]:
#!kill 3285
#%tensorboard --logdir runs

In [23]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [27]:
!nvidia-smi

Tue May 25 22:44:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    39W / 300W |   2355MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  lstmpredictions.png


In [25]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('trainlstmpredictions.png',dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  trainlstmpredictions.png


In [26]:
for name, param in model.named_parameters():
  print(name)

lstm_cell.weight_ih_l0
lstm_cell.weight_hh_l0
lstm_cell.bias_ih_l0
lstm_cell.bias_hh_l0
lstm_cell.weight_ih_l1
lstm_cell.weight_hh_l1
lstm_cell.bias_ih_l1
lstm_cell.bias_hh_l1
inject_image.0.weight
inject_image.0.bias
fc_out.0.weight
fc_out.0.bias
fc_out.3.weight
fc_out.3.bias
hi_1.0.weight
hi_1.0.bias
hi_2.0.weight
hi_2.0.bias
ci_1.0.weight
ci_1.0.bias
ci_2.0.weight
ci_2.0.bias
alpha_1.0.weight
alpha_1.0.bias
alpha_long1.0.weight
alpha_long1.0.bias
